In [136]:
import pandas as pd
import ssl
import numpy as np
from pathlib import Path
import sqlalchemy
import datetime

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

database_connection_string = 'sqlite:///SP500.db'
engine = sqlalchemy.create_engine(database_connection_string)

In [137]:
SectorDF = pd.read_sql_table('SectorDF', engine)
SectorDF = SectorDF.drop(columns=['dateTime'])
SectorDF

,SPY,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy
0,0.001898,0.004364,0.001239,0.000170,0.002117,0.006057,0.001120,-0.000095,0.006061,0.002487,0.002266,-0.017691
1,0.003134,-0.001641,0.006489,0.000045,-0.000767,0.000146,-0.000002,0.016096,-0.004232,-0.003210,0.007684,0.001140
2,-0.003415,-0.000804,0.000607,-0.004617,-0.004491,-0.001597,-0.002539,0.007733,0.010694,0.009282,-0.004979,0.020342
3,0.005027,0.000453,0.011968,0.004855,0.003417,0.005316,-0.004535,0.011355,-0.001082,0.004324,0.011913,-0.006888
4,-0.000072,-0.003498,0.002734,-0.002692,-0.003881,0.001754,0.002421,0.002735,-0.004246,-0.001499,-0.003487,-0.018957
...,...,...,...,...,...,...,...,...,...,...,...,...
250,-0.000783,0.002059,0.011836,-0.003392,-0.005008,0.009086,-0.006456,0.019110,-0.000604,-0.004417,0.014811,-0.010703
251,0.006956,0.005856,0.020828,0.005208,-0.009337,0.008555,0.004695,-0.003217,-0.001480,0.004416,-0.004456,0.002626
252,-0.001049,-0.005791,0.005329,-0.007141,0.001863,0.001890,0.004482,0.001318,0.002899,0.002020,0.001696,0.027976
253,-0.009435,-0.001636,-0.016198,-0.008222,-0.001345,-0.002333,0.006947,-0.014041,-0.002284,-0.001462,-0.010802,-0.018648


In [138]:
# Define the target set y using the SPY column
y = SectorDF["SPY"]

# Display a sample of y
y[:5]

0    0.001898
1    0.003134
2   -0.003415
3    0.005027
4   -0.000072
Name: SPY, dtype: float64

In [139]:
# Define features set X by selecting all columns but SPY
X = SectorDF.drop(columns=["SPY"])

# Review the features DataFrame
X.head()

,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy
0,0.004364,0.001239,0.000170,0.002117,0.006057,0.001120,-0.000095,0.006061,0.002487,0.002266,-0.017691
1,-0.001641,0.006489,0.000045,-0.000767,0.000146,-0.000002,0.016096,-0.004232,-0.003210,0.007684,0.001140
2,-0.000804,0.000607,-0.004617,-0.004491,-0.001597,-0.002539,0.007733,0.010694,0.009282,-0.004979,0.020342
3,0.000453,0.011968,0.004855,0.003417,0.005316,-0.004535,0.011355,-0.001082,0.004324,0.011913,-0.006888
4,-0.003498,0.002734,-0.002692,-0.003881,0.001754,0.002421,0.002735,-0.004246,-0.001499,-0.003487,-0.018957


In [140]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [141]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [142]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

11

In [153]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [154]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  100 #(number_input_features + 1) // 2 

# Review the number hidden nodes in the first layer
hidden_nodes_layer1

100

In [155]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 50 #(hidden_nodes_layer1 + 1) // 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2

50

In [156]:
# Create the Sequential model instance
nn = Sequential()

In [157]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

In [158]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="tanh"))

In [159]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))

In [160]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 100)               1200      
                                                                 
 dense_27 (Dense)            (None, 50)                5050      
                                                                 
 dense_28 (Dense)            (None, 1)                 51        
                                                                 
Total params: 6,301
Trainable params: 6,301
Non-trainable params: 0
_________________________________________________________________


In [161]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [162]:
# Fit the model using 500 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=500)

Epoch 1/500
6/6 [==============================] - 0s 1ms/step - loss: 0.6998 - accuracy: 0.0052
Epoch 2/500
6/6 [==============================] - 0s 1ms/step - loss: 0.6354 - accuracy: 0.0052
Epoch 3/500
6/6 [==============================] - 0s 1ms/step - loss: 0.5694 - accuracy: 0.0052
Epoch 4/500
6/6 [==============================] - 0s 2ms/step - loss: 0.5121 - accuracy: 0.0052
Epoch 5/500
6/6 [==============================] - 0s 1ms/step - loss: 0.4515 - accuracy: 0.0052
Epoch 6/500
6/6 [==============================] - 0s 1ms/step - loss: 0.3932 - accuracy: 0.0052
Epoch 7/500
6/6 [==============================] - 0s 2ms/step - loss: 0.3341 - accuracy: 0.0052
Epoch 8/500
6/6 [==============================] - 0s 2ms/step - loss: 0.2742 - accuracy: 0.0052
Epoch 9/500
6/6 [==============================] - 0s 2ms/step - loss: 0.2169 - accuracy: 0.0052
Epoch 10/500
6/6 [==============================] - 0s 2ms/step - loss: 0.1674 - accuracy: 0.0052
Epoch 11/500
6/6 [===========

In [163]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: -2.0519e-01 - accuracy: 0.0000e+00 - 90ms/epoch - 45ms/step
Loss: -0.20518779754638672, Accuracy: 0.0
